Поработаем над датасетом из соревнования [UtkMl](https://www.kaggle.com/competitions/utkmls-twitter-spam-detection-competition/overview) про определение спама в твитах.  

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn import metrics

from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('train.csv')
data.drop('Unnamed: 7', axis=1, inplace=True)  
data.head()

Почистим датасет

In [ ]:
data.info()

In [ ]:
data.Type.unique()

In [ ]:
data = data[data.Type != "South Dakota"]

In [ ]:
data['Type'] = data['Type'].apply(lambda x: 0 if x == 'Quality' else 1)

In [ ]:
data.drop('location', axis=1, inplace=True)

In [ ]:
y = data['Type']
X = data.drop('Type', axis=1)

Давайте построим графики. 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
for ax, feat in zip(axes.flat, X.select_dtypes(include=np.number).columns):
    sns.kdeplot(X[feat], ax=ax)
plt.show()

Видим, что а) ни один из признаков не имеет нормального распределения б) для небинарных признаков самое частое значение - 0. Скорее всего, наиболее важную информацию несут сами твиты (мы намеренно не будем удалять ссылки и хештеги, потому что они, скорее всего, вообще сделают самый важный вклад).

Теперь нам необходимо обработать наши признаки. Часть из них - текстовые, и их нужно обработать отдельно. 

In [ ]:
def feature_engineering(choice_transformer, choice_ngrams):
    # числовые характеристики нормализуем: imputer обрабатывает наны, scaler масштабирует
    numeric_features = ['following', 'followers', 'actions', 'is_retweet']
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    )

    # текстовые характеристики обрабатываем: либо tf-idf, либо мешок слов
    text_features = 'Tweet'
    if choice_transformer == 'tfidf':
        text_transformer = TfidfVectorizer(ngram_range=choice_ngrams, tokenizer=word_tokenize, stop_words='english')
    else:
        text_transformer = CountVectorizer(ngram_range=choice_ngrams, tokenizer=word_tokenize, stop_words='english')

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("txt", text_transformer, text_features),
        ]
    )
    return preprocessor

Обучим модели.

In [ ]:
test = pd.read_csv('test.csv')
test.drop('Id', axis=1, inplace=True)
test.head()

In [ ]:
def modelfit(model):
    model.fit(Xtrain, ytrain)
    
    ypredtest = model.predict(Xtest)
    ypredtrain = model.predict(Xtrain)
    
    print(accuracy_score(ytest, ypredtest), accuracy_score(ytrain, ypredtrain))

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2)

TF-IDF unigrams

In [ ]:
preprocessor = feature_engineering('tfidf', (1, 1))

clfLR = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

clfSVC = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)

In [ ]:
modelfit(clfLR)
modelfit(clfSVC)

TF-IDF bigrams

In [ ]:
preprocessor = feature_engineering('tfidf', (2, 2))

clfLR = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

clfSVC = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)

In [ ]:
modelfit(clfLR)
modelfit(clfSVC)

BOW unigrams

In [ ]:
preprocessor = feature_engineering('bow', (1, 1))

clfLR = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

clfSVC = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)

In [ ]:
modelfit(clfLR)
modelfit(clfSVC)

BOW bigrams

In [ ]:
preprocessor = feature_engineering('bow', (2, 2))

clfLR = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

clfSVC = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)

In [ ]:
modelfit(clfLR)
modelfit(clfSVC)

BOW Bagging

In [ ]:
preprocessor = feature_engineering('bow', (1, 1))

bagging = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", BaggingClassifier())]
)

In [ ]:
modelfit(bagging)

In [ ]:
preprocessor = feature_engineering('bow', (2, 2))

bagging = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", BaggingClassifier())]
)

In [ ]:
modelfit(bagging)

Random Forest

In [ ]:
preprocessor = feature_engineering('bow', (2, 2))

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

modelfit(clf)

Проверка только на твитах (чтобы посмотреть, что влияет сильнее всего)

In [ ]:
vec = CountVectorizer(ngram_range=(3, 3), tokenizer=word_tokenize, stop_words='english')
bow = vec.fit_transform(Xtrain['Tweet'])
clf = DecisionTreeClassifier()
clf.fit(bow, ytrain)
ypredtest = clf.predict(vec.transform(Xtest['Tweet']))
print(classification_report(ypredtest, ytest))

In [ ]:
list(vec.vocabulary_.items())[:10]

Проверим гипотезу о том, что ссылки сильнее всего влияют на определение спама

In [ ]:
tweets = list(data['Tweet'])
nums = []

for num, w in enumerate(tweets):
    if 't.co' in w:
        nums.append(num)

In [ ]:
spamham = list(data['Type'])
needed = []
for i, w in enumerate(spamham):
    if i in nums:
        needed.append(w)

In [ ]:
from collections import Counter

Counter(needed)

Оказывается, подавляющее большинство текстов со ссылками определено как спам.

А теперь давайте посмотрим, как можно было ту же задачу решить с помощью эмбеддингов Doc2Vec из gensim.

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Doc2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
from tqdm import tqdm

In [ ]:
gensim.__version__

In [ ]:
import nltk
from nltk.corpus import stopwords

def tokenize_text(text):
    '''gensim сам токенизировать не умеет, поэтому нам придется сделать это за него'''
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

train, test = train_test_split(data[['Tweet', 'Type']], test_size=0.3, random_state=42)

# соберем специальный объект класса TaggedDocument, чтобы D2V работал
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Tweet']), tags=[r.Type]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['Tweet']), tags=[r.Type]), axis=1)

Воспользуемся тем, что у большинства современных процессоров больше одного ядра...

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

Обучим модельку DBoW (Distributed Bag of Words)

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample=0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

In [ ]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

Напишем аналог transform для D2V:

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

Обучим банальную логистическую регрессию

In [ ]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
logreg = LogisticRegression(solver='liblinear', n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

С другими алгоритмами можете побаловаться сами. 

### Задание

Попробуйте поработать с датасетом юридических текстов. В датасете всего две важных колонки признаков: заголовок дела и его текст, а целевая переменная - case_outcome (мультиклассовая классификация). 

В базовом варианте можно оставить только текст дела, если хотите поинтереснее - можно попробовать распарсить case_title, добыв оттуда дополнительные признаки. 

In [27]:
data = pd.read_csv('legal_text_classification.csv')
data.head()

,case_id,case_outcome,case_title,case_text
0,Case1,cited,Alpine Hardwood (Aust) Pty Ltd v Hardys Pty Lt...,Ordinarily that discretion will be exercised s...
1,Case2,cited,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...
2,Case3,cited,Colgate Palmolive Co v Cussons Pty Ltd (1993) ...,Ordinarily that discretion will be exercised s...
3,Case4,cited,Dais Studio Pty Ltd v Bullett Creative Pty Ltd...,The general principles governing the exercise ...
4,Case5,cited,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...
